# Imports

In [ ]:
video_path = "V1.mp4"

In [1]:
import os

from textwrap import dedent
import moviepy.editor as mp
import speech_recognition as sr

from langchain_groq import ChatGroq
from crewai import Agent, Task, Process, Crew


# Functions

## Getting Text From Audio

In [2]:
# Function to transcribe video to text
def transcribe_video(video_path):
    recognizer = sr.Recognizer()
    
    # Extract audio from video
    video = mp.VideoFileClip(video_path)
    audio_path = "temp_audio.wav"
    video.audio.write_audiofile(audio_path)
    
    transcript = ""
    try:
        with sr.AudioFile(audio_path) as source:
            audio = recognizer.record(source)
        # Transcribe audio to text
        transcript = recognizer.recognize_google(audio)
    except sr.UnknownValueError:
        transcript = "Google Speech Recognition could not understand audio"
    except sr.RequestError as e:
        transcript = f"Could not request results from Google Speech Recognition service; {e}"
    finally:
        # Clean up the temporary audio file
        if os.path.exists(audio_path):
            os.remove(audio_path)
    
    return transcript

## Getting captions from LLM

In [ ]:
GROQ_API_KEY = "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"
Text_Input = """I love you Grand rising good Vibes to all status I'm ok happy Monday Happy Tuesday happy Wednesday happy Thursday happy Friday and a good New Year everyone"""
LLM_Model = "llama3-70b-8192"
def create_gif_caption_identifier_task(api_key, model, text_input):
    
    # Initialize the LLM
    LLM_Grouq = ChatGroq(
        api_key=api_key,
        model=model
    )

    # Create the GIF Caption Finder agent
    GIF_caption_identifier = Agent(
        role="GIF Caption Finder",
        goal="Identify sentences that can be used as text over GIFs",
        backstory=dedent("""\
            You're a GIF caption expert tasked with identifying sentences that are ideal for adding text overlays to GIFs. 
            Your role is crucial in selecting text fragments that convey impactful and memorable messages to enhance the visual appeal and communication of the GIFs. 
            Your task is to carefully read the provided text and list all sentences that are suitable for GIF captions, ensuring they are concise, relevant, and engaging.
        """),
        verbose=True,
        allow_delegation=True,
        llm=LLM_Grouq,
        memory=True,
    )

    # TASK
    GIF_caption_identifier_Task = Task(
        description=dedent(f"""\
            **INPUT:**
                * **Text:**
                    {text_input}
            **TASK:**
                Identify sentences within the text that can be used as captions for GIFs. ENSURE THE SELECTED SENTENCES ARE EXACTLY AS IN THE INPUT TEXT. NO SINGLE WORD OR LETTER SHOULD BE DIFFERENT OR NEW, and the sentences should be impactful and contextually appropriate for a GIF.

            **OUTPUT:**
                Output the selected sentences as a JSON object, with each sentence being a separate entry.
            
            **EXAMPLE OUTPUT:**
                {{
                    "captions": [
                        "This is a perfect moment!",
                        "I can't believe this is happening.",
                        "Let's make this unforgettable."
                    ]
                }}
        """),
        agent=GIF_caption_identifier,
        expected_output=dedent("""\
            {
                "captions": [
                    "Sentence 1",
                    "Sentence 2",
                    ...
                    "Sentence n"
                ]
            }
        """),
        async_execution=False
    )

    crew = Crew(
        agents=[GIF_caption_identifier],
        tasks=[GIF_caption_identifier_Task],
        verbose=2,
        process=Process.sequential,
    )

    return crew


# Test

In [3]:
print(transcribe_video("V1.mp4"))

MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.
I love you Grand rising good Vibes to all status I'm ok happy Monday Happy Tuesday happy Wednesday happy Thursday happy Friday and a good New Year everyone
